In [1]:
# 이미지 노이즈 효과 적용 및 저장

from google.colab import files
from PIL import Image
import numpy as np
import os
import ipywidgets as widgets
from IPython.display import display, HTML

# 파일 업로드
uploaded = files.upload()

image_path = list(uploaded.keys())[0]
input_format = os.path.splitext(image_path)[1][1:].upper()

original_image = Image.open(image_path).convert('RGB')

# Gaussian 노이즈 추가 함수
def add_gaussian_noise(image, intensity=0.1):
    np_image = np.array(image, dtype=np.float32)
    mean = 0
    stddev = intensity * 255
    noise = np.random.normal(mean, stddev, np_image.shape)
    noisy_image = np.clip(np_image + noise, 0, 255)
    return Image.fromarray(noisy_image.astype(np.uint8))

# Salt-and-Pepper 노이즈 추가 함수
def add_salt_and_pepper_noise(image, intensity=0.05):
    np_image = np.array(image, dtype=np.uint8)
    prob = intensity
    noisy_image = np_image.copy()

    num_salt = np.ceil(prob * np_image.size * 0.5)
    num_pepper = np.ceil(prob * np_image.size * 0.5)

    coords_salt = [np.random.randint(0, i, int(num_salt)) for i in np_image.shape[:2]]
    noisy_image[coords_salt[0], coords_salt[1], :] = 255

    coords_pepper = [np.random.randint(0, i, int(num_pepper)) for i in np_image.shape[:2]]
    noisy_image[coords_pepper[0], coords_pepper[1], :] = 0

    return Image.fromarray(noisy_image)

# Gaussian 노이즈 추가
gaussian_noisy_image = add_gaussian_noise(original_image, intensity=0.1)

# Salt-and-Pepper 노이즈 추가
salt_and_pepper_noisy_image = add_salt_and_pepper_noise(original_image, intensity=0.01)

# Gaussian + Salt-and-Pepper 결합된 노이즈 이미지
combined_noisy_image = add_salt_and_pepper_noise(gaussian_noisy_image, intensity=0.02)

# 이미지 비교
import matplotlib.pyplot as plt

plt.figure(figsize=(18, 6))

plt.subplot(1, 4, 1)
plt.imshow(original_image)
plt.title("Original Image")
plt.axis("off")

plt.subplot(1, 4, 2)
plt.imshow(gaussian_noisy_image)
plt.title("Gaussian Noise")
plt.axis("off")

plt.subplot(1, 4, 3)
plt.imshow(salt_and_pepper_noisy_image)
plt.title("Salt-and-Pepper Noise")
plt.axis("off")

plt.subplot(1, 4, 4)
plt.imshow(combined_noisy_image)
plt.title("Combined Noise")
plt.axis("off")

plt.show()

first_prompt = True

save_option_widget = widgets.RadioButtons(
    options=['Y', 'N'],
    description='저장 여부:',
    disabled=False,
    value=None
)

previous_save_choice = None
image_choice_widget = None

# 저장 여부 선택 함수
def on_save_option_change(change):
    global first_prompt, previous_save_choice, image_choice_widget

    if change['new'] == 'Y':
        previous_save_choice = 'Y'
        save_image(first_prompt=True)
    elif change['new'] == 'N':
        previous_save_choice = 'N'

        save_option_widget.disabled = True
        if image_choice_widget:
            image_choice_widget.disabled = True
        print("저장 프로세스를 종료합니다.")

        return
    elif change['new'] is None:
        if previous_save_choice is None:
            save_option_widget.value = None

# 이미지 저장 함수
def save_image(first_prompt=False):
    global input_format, image_choice_widget

    if first_prompt:
        image_choice_widget = widgets.RadioButtons(
            options=['Gaussian Noise', 'Salt-and-Pepper Noise', 'Combined Noise'],
            description='노이즈 방식:',
            disabled=False,
            value=None
        )
        display(image_choice_widget)

        # 이미지 선택 처리
        def on_image_choice_change(change):
            if change['new'] is not None:
                save_name_widget = widgets.Text(
                    value='',
                    placeholder='파일 이름 입력',
                    description='파일 이름:',
                    disabled=False
                )

                save_button = widgets.Button(description="저장")

                save_button.style.button_color = 'blue'
                save_button.style.font_weight = 'bold'

                save_button.add_class('btn-white-text')

                ui = widgets.HBox([save_name_widget, save_button])
                display(ui)

                # 저장 버튼 클릭 시 처리 함수
                def on_save_button_click(b):
                    global input_format
                    save_name = save_name_widget.value.strip()
                    if save_name == '':
                        print("파일 이름을 입력해야 합니다.")
                        return
                    image_choice = change['new']

                    # 선택한 이미지에 따라 저장 처리
                    if image_choice == 'Gaussian Noise':
                        selected_image = gaussian_noisy_image
                    elif image_choice == 'Salt-and-Pepper Noise':
                        selected_image = salt_and_pepper_noisy_image
                    elif image_choice == 'Combined Noise':
                        selected_image = combined_noisy_image

                    if input_format == "JPG":
                        input_format = "JPEG"

                    # 파일 경로 설정
                    file_path = f"{save_name}.{input_format.lower()}"

                    # 로컬 파일 저장
                    selected_image.save(file_path, format=input_format)
                    print(f"{file_path} 파일이 저장되었습니다. \n")

                    files.download(file_path)

                    save_option_widget.value = None
                    additional_save_option()

                save_button.on_click(on_save_button_click)

        image_choice_widget.observe(on_image_choice_change, names='value')

    else:
        additional_save_option()

def additional_save_option():
    print("\n추가 저장 여부를 선택하세요.")
    display(save_option_widget)

# 초기값을 None으로 설정한 후에 observe 함수 실행
save_option_widget.observe(on_save_option_change, names='value')
display(save_option_widget)

display(HTML("<style>.btn-white-text { color: white; }</style>"))


ModuleNotFoundError: No module named 'google.colab'